# Starting our Docker Environment

In [ ]:
!./gradlew --console=plain tasks --group docker -q

In [ ]:
!./gradlew --console=plain -q composeUp

In [ ]:
ksql = "/usr/local/bin/ksql"
!echo "LIST TOPICS;" | "$ksql"

# The Developer Experience
Let's create a table called `CLICKSTREAM`. This is a *registration query*... it's like a DDL statement:

### Create stream CLICKSTREAM

In [ ]:
sql = (
    "CREATE STREAM clickstream "
    "(_time bigint, time varchar, ip varchar, request varchar, status int, userid int, bytes bigint, agent varchar) "
    "with (kafka_topic = 'clickstream', value_format = 'json');"
)
!echo "$sql" | "$ksql"

### Create table EVENTS_PER_MIN

Now, let's create create another table called `EVENTS_PER_MIN` that uses `CLICKSTREAM`. This is a *persistent query*... it selects data from an existing KSQL stream or table, creates or uses an underlying Kafka topic as a target and initializes the streaming process to transform and persist data to that topic:

In [ ]:
sql = (
    "CREATE TABLE events_per_min AS SELECT userid, count(*) AS events "
    "FROM clickstream window "
    "TUMBLING (size 60 second) GROUP BY userid;"
)
!echo "$sql" | "$ksql"

### Iterating on our Pipeline
Now, I want to make some changes to `CLICKSTREAM`. I'll just drop it right?
### Drop stream CLICKSTREAM

In [ ]:
sql = (
    "DROP STREAM clickstream; "
)
!echo "$sql" | "$ksql"

### Terminate our auto-incrementing query id

In [ ]:
sql = (
    "TERMINATE QUERY CTAS_EVENTS_PER_MIN_0; "
)
!echo "$sql" | "$ksql"

### Now drop CLICKSTREAM again

In [ ]:
sql = (
    "DROP STREAM clickstream; "
)
!echo "$sql" | "$ksql"

### Is this workable long-term?
Wouldn't it be great if this could be automated?

# Stopping our Docker environment

In [ ]:
!./gradlew --console=plain -q composeDown